In [ ]:
 # I will use until setting up the database
# from pymongo import MongoClient

# client = MongoClient("mongodb://localhost:27017/")
# db = client["my_agent_db"]
# collection = db["conversation_memory"]

## Loading APIs

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

## Setup User's Profile

In [2]:
profile = {
    "name": "Valentin",
    "full_name": "Valentin Jimenez",
    "user_profile_background": "Student using AI tools to leverage learning",
}

#  Prompt instructions and email example

In [ ]:
prompt_classification_instructions = {
    "email_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements.",
        "notify": "Job offers, job listings, and company announcements realted to AI agents",
        "respond": "Direct messages from recruiters, customer inquiries, and urgent project requests.",
    },
    "agent_instructions": "Use these tools when appropriate to help manage Valentin's tasks efficiently."
}

In [4]:
# Example incoming email, as part of initial few shot prompting
email_example_1 = {
    "from": "Victor Alejandro <victor.alejandro@company.com>",
    "to": "Valentin Jimenez <santi.job.2022@gmail.com>",
    "subject": "Quick question about your expierience with AI agents frameworks",
    "body": """
Hi Valentin,

We have a project where we think we can leverage the power of AI to improve our product. 
I was wondering if you have any experience with Langraph or Crew AI frameworks?

Hope you cant take a look as soon as possible since we are close to the deadline.

Thanks!
Victor""",
}

## Define response system

During prototype face I will use a mock email service. Later I need to use a real email service.

In [5]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated
from langchain.chat_models import init_chat_model

In [6]:
llm = init_chat_model("openai:gpt-4o-mini")

Using pydantic to generate a specified structured output

In [7]:
class Router(BaseModel):
    """Analyze the new unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [8]:
llm_router = llm.with_structured_output(Router)

In [9]:
llm_router

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000020F969BA510>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000020F969BAF90>, root_client=<openai.OpenAI object at 0x0000020F9684CAD0>, root_async_client=<openai.AsyncOpenAI object at 0x0000020F969BA660>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.Router'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'Router', 'description': 'Analyze the new unread email and route it according to its content.', 'parameters': {'properties': {'reasoning': {'description': 'Step-by-step reasoning behind the classification.', 'type': 'string'}, 'classification': {'description': "The classification of an email: 'ignore' for irrelevant emails, 'notify' for important information

In [10]:
from prompts import email_classification_system_prompt, email_user_prompt

In [11]:
system_prompt = email_classification_system_prompt.format(
    full_name=profile["full_name"],
    name=profile["name"],
    examples=None,
    user_profile_background=profile["user_profile_background"],
    email_no=prompt_classification_instructions["email_rules"]["ignore"],
    email_notify=prompt_classification_instructions["email_rules"]["notify"],
    email_respond=prompt_classification_instructions["email_rules"]["respond"],
)

In [12]:
user_prompt = email_user_prompt.format(
    author=email_example_1["from"],
    to=email_example_1["to"],
    subject=email_example_1["subject"],
    email_thread=email_example_1["body"],
)

In [13]:
result = llm_router.invoke(
    [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
)

In [14]:
# Here maybe use another agent to produce more emails and test the system message, trace with opik for instance??
# fine tune with emails classification?
print(result)

reasoning="This email is a direct inquiry from Victor Alejandro about Valentin's experience with specific AI agents frameworks, showing urgency due to a deadline mentioned. It requires a direct response to provide the requested information." classification='respond'


## Main agent: Tools definition

In [15]:
from langchain_core.tools import tool

In [16]:
# Here is where the connection with the API must be made
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder
    return f"Email sent to {to} with subject '{subject}'"

In [18]:
# This one too, connect with calendars API's
@tool
def schedule_meeting(
    attendees: list[str], 
    subject: str, 
    duration_minutes: int, 
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"

In [29]:
@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available time on {day}: 9:00 AM"

## Main agent: initial system prompt creation

In [30]:
from prompts import agent_system_prompt
def create_prompt(state):
    return [
        {
            "role": "system", 
            "content": agent_system_prompt.format(
                instructions=prompt_classification_instructions["agent_instructions"],
                **profile
                )
        }
    ] + state['messages']

In [31]:
print(agent_system_prompt)


< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about managing {name}'s job applications.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
</ Tools >

< Instructions >
{instructions}
</ Instructions >



## Using the prebuil create_react_agent

This will be a subagent deciding what tool to use according to the received email

In [32]:
from langgraph.prebuilt import create_react_agent

In [33]:
tools=[write_email, schedule_meeting, check_calendar_availability]

In [34]:
agent = create_react_agent(
    "openai:gpt-4o-mini",
    tools=tools,
    prompt=create_prompt,
)

Testing the create_react_agent agent. Note that this has not the first defined structured output

In [35]:
response = agent.invoke(
    {"messages": [{
        "role": "user", 
        "content": "what is my availability for tuesday?"
    }]}
)

In [36]:
response["messages"][-1].pretty_print()

================================== Ai Message ==================================

You are available at 9:00 AM on Tuesday.
